In [1]:
import pandas as pd

# Trading function

In [2]:
last_training = 621.38

def trading(df, n_ahead):
    adjusted_n_ahead = n_ahead-1 # bcuz lists start at 0
    df_adj = df.iloc[adjusted_n_ahead::63]

    latest_decision = 'hold'
    bank = 1000
    stock_count = 0

    for row, values in df_adj.iterrows():
        latest_close = last_training if values['time'] == 1 else df['Close'].iloc[row-adjusted_n_ahead-63]
        forecast = df['forecast'].iloc[row]
        expected = forecast - latest_close
        
        decision = 'hold'

        if expected > 0 and latest_decision != 'buy':
            decision = 'buy'
            latest_decision = 'buy'
            bank -= latest_close
            stock_count += 1
        
        if expected < 0 and latest_decision != 'sell' and stock_count > 0:
            decision = 'sell'
            latest_decision = 'sell'
            bank += latest_close
            stock_count -= 1
        
        # print(decision, forecast, latest_close, bank)
    
    last_close = df['Close'].iloc[-63]
    if latest_decision == 'buy': # reverts decision if latest decision is buy
        bank += last_close

    buy_hold = last_close - last_training
    trading = bank - 1000

    print(f'trading: {round(trading, 2)}')
    print(f'buy&hold: {round(buy_hold, 2)} (buy: {last_training}, sell: {last_close})')

# ARMA-GARCH-trading

In [3]:
# reading data
ag_df = pd.read_csv('data/r_cross_val.csv', usecols=[0,1,2])

In [4]:
trading(ag_df, 1)

trading: 732.61
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [5]:
trading(ag_df,3)

trading: 727.35
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [6]:
trading(ag_df,5)

trading: 699.91
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [7]:
trading(ag_df, 21)

trading: 573.33
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [8]:
trading(ag_df, 63)

trading: 368.5
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


# LSTM-trading

In [9]:
lstm_df = pd.read_csv('data/lstm_results.csv', usecols=[0,1,2])

In [10]:
trading(lstm_df, 1)

trading: 77.84
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [11]:
trading(lstm_df, 3)

trading: -2.6
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [12]:
trading(lstm_df, 5)

trading: 47.79
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [13]:
trading(lstm_df, 21)

trading: 150.27
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)


In [14]:
trading(lstm_df, 63)

trading: 51.33
buy&hold: 384.64 (buy: 621.38, sell: 1006.02)
